<a href="https://colab.research.google.com/github/smriti09/deep_learning/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


FOLDERNAME = 'Project/Project/DeepTraffic-master'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv1D, Conv2D,MaxPool1D, MaxPool2D

In [ ]:
!pip install idx2numpy

In [ ]:
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

In [ ]:
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
from sklearn.model_selection import KFold
get_custom_objects().update({'swish': Activation(swish)})

In [ ]:
#############defining metrics###########

def recall(y_true, y_pred):
    
    true_positives = Kb.sum(Kb.round(Kb.clip(y_true * y_pred, 0, 1)))
    all_positives = Kb.sum(Kb.round(Kb.clip(y_true, 0, 1)))
    
    recall = true_positives / (all_positives + Kb.epsilon())
    return recall

def precision(y_true, y_pred):
    
    true_positives = Kb.sum(Kb.round(Kb.clip(y_true * y_pred, 0, 1)))
    
    predicted_positives = Kb.sum(Kb.round(Kb.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + Kb.epsilon())
    return precision

In [ ]:
##########download dataset################  FLOWALLLAYERS 2CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/2class/FlowAllLayers
!pwd
import matplotlib.pyplot as plt
import numpy as np
import os
import idx2numpy
imagefile = 'train-image-idx3-ubyte'
x_train = idx2numpy.convert_from_file(imagefile)
x_test = idx2numpy.convert_from_file('t10k-images-idx3-ubyte')
y_train = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y_test = idx2numpy.convert_from_file('t10k-labels-idx1-ubyte')
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/2class/FlowAllLayers
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/2class/FlowAllLayers
(41822, 28, 28)
(4647, 28, 28)
(41822,)
(4647,)


In [ ]:

############CUSTOM LOSS for 2 class###############
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=2
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/41822)
  
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,784,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,784,1])
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 784, 32)           832       
                                                                 
 activation_1 (Activation)   (None, 784, 32)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 262, 32)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 262, 64)           51264     
                                                                 
 activation_2 (Activation)   (None, 262, 64)           0         
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 88, 64)           0         
 1D)                                                    

[0.032497163861989975, 0.3593716323375702, 1.35659921169281, 0.385038286447525]

In [ ]:

######## 1DCNN ########### 5 fold cross validation 
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,784,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,784,1])
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
#model.add(Dropout(0.2))
model.add(Dense(2))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 784, 32)           832       
                                                                 
 activation_3 (Activation)   (None, 784, 32)           0         
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 262, 32)          0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 262, 64)           51264     
                                                                 
 activation_4 (Activation)   (None, 262, 64)           0         
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 88, 64)           0         
 1D)                                                  

[0.03258949890732765, 0.3593716323375702, 1.0, 0.35940051078796387]

In [ ]:
############## 2DCNN ###################### 5 fold cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(2))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        832       
                                                                 
 activation_7 (Activation)   (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 64)        51264     
                                                                 
 activation_8 (Activation)   (None, 10, 10, 64)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                  

[0.03266298025846481, 0.6406283378601074, 0.0, 0.0]

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(2))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 28, 28, 32)        832       
                                                                 
 activation_9 (Activation)   (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 10, 10, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 10, 10, 64)        51264     
                                                                 
 activation_10 (Activation)  (None, 10, 10, 64)        0         
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                  

[0.03254079073667526, 0.6406283378601074, 1.0, 0.35940051078796387]

In [ ]:
##########download dataset################  FLOWL7 2CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/2class/FlowL7
!pwd
x2_train_fl7 = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x2_test_fl7 = idx2numpy.convert_from_file('t10k-images-idx3-ubyte')
y2_train_fl7 = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y2_test_fl7 = idx2numpy.convert_from_file('t10k-labels-idx1-ubyte')
print(x2_train_fl7.shape)
print(x2_test_fl7.shape)
print(y2_train_fl7.shape)
print(y2_test_fl7.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/2class/FlowL7
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/2class/FlowL7
(34851, 28, 28)
(3872, 28, 28)
(34851,)
(3872,)


In [ ]:
############CUSTOM LOSS for 2 class############### FLOW L7

def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=2
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/34851)
  print("model_loss:",custom_loss)
  
  #custom_loss=Kb.square(y_true-y_pred)
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x2_train_fl7.shape[0]
x_train=tf.reshape(x2_train_fl7,[N,784,1])
N2=x2_test_fl7.shape[0]
x_test=tf.reshape(x2_test_fl7,[N2,784,1])
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y2_train_fl7,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y2_test_fl7,batch_size=batch_size,verbose=2)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 784, 32)           832       
                                                                 
 activation_5 (Activation)   (None, 784, 32)           0         
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 262, 32)          0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 262, 64)           51264     
                                                                 
 activation_6 (Activation)   (None, 262, 64)           0         
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 88, 64)           0         
 1D)                                                  

[0.03253522887825966,
 0.41322314739227295,
 0.37758970260620117,
 0.5460605621337891]

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x2_train_fl7.shape[0]
x_train=tf.reshape(x2_train_fl7,[N,784,1])
N2=x2_test_fl7.shape[0]
x_test=tf.reshape(x2_test_fl7,[N2,784,1])
y_train=y2_train_fl7
y_test= y2_test_fl7
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.0009)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 784, 32)           832       
                                                                 
 activation_7 (Activation)   (None, 784, 32)           0         
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 262, 32)          0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 262, 64)           51264     
                                                                 
 activation_8 (Activation)   (None, 262, 64)           0         
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 88, 64)           0         
 1D)                                                  

ValueError: ignored

In [ ]:
N=x2_train_fl7.shape[0]
x_train=tf.reshape(x2_train_fl7,[N,28,28,1])
N2=x2_test_fl7.shape[0]
x_test=tf.reshape(x2_test_fl7,[N2,28,28,1])
y_train=y2_train_fl7
y_test= y2_test_fl7
model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(2))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 activation_9 (Activation)   (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 10, 10, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 64)        51264     
                                                                 
 activation_10 (Activation)  (None, 10, 10, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                  

[0.038998447358608246, 0.586776852607727, 0.0, 0.0]

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x2_train_fl7.shape[0]
x_train=tf.reshape(x2_train_fl7,[N,28,28,1])
N2=x2_test_fl7.shape[0]
x_test=tf.reshape(x2_test_fl7,[N2,28,28,1])
y_train=y2_train_fl7
y_test= y2_test_fl7
model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(2))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        832       
                                                                 
 activation_11 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 64)        51264     
                                                                 
 activation_12 (Activation)  (None, 10, 10, 64)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                  

[0.03894054889678955, 0.586776852607727, 0.0, 0.0]

In [ ]:
##########download dataset################  SESSIONALLLAYERS 2CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/2class/SessionAllLayers
!pwd
x2_train_s = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x2_test_s = idx2numpy.convert_from_file('t10k-images-idx3-ubyte')
y2_train_s = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y2_test_s = idx2numpy.convert_from_file('t10k-labels-idx1-ubyte')
print(x2_train_s.shape)
print(x2_test_s.shape)
print(y2_train_s.shape)
print(y2_test_s.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/2class/SessionAllLayers
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/2class/SessionAllLayers
(35501, 28, 28)
(3945, 28, 28)
(35501,)
(3945,)


In [ ]:
############CUSTOM LOSS for 2 class############### SESSION ALL
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=2
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/35501)
  
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x2_train_s.shape[0]
x_train=tf.reshape(x2_train_s,[N,784,1])
N2=x2_test_s.shape[0]
x_test=tf.reshape(x2_test_s,[N2,784,1])
y_train=y2_train_s
y_test=y2_test_s
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_10 (Conv1D)          (None, 784, 32)           832       
                                                                 
 activation_15 (Activation)  (None, 784, 32)           0         
                                                                 
 max_pooling1d_10 (MaxPoolin  (None, 262, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_11 (Conv1D)          (None, 262, 64)           51264     
                                                                 
 activation_16 (Activation)  (None, 262, 64)           0         
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 88, 64)           0         
 g1D)                                                 

[0.03892141953110695,
 0.6823827624320984,
 1.0508623123168945,
 0.26745179295539856]

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x2_train_s.shape[0]
x_train=tf.reshape(x2_train_s,[N,784,1])
N2=x2_test_s.shape[0]
x_test=tf.reshape(x2_test_s,[N2,784,1])
y_train=y2_train_s
y_test=y2_test_s
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
x=x_train
y=y_train
# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
#model.add(Dropout(0.2))
model.add(Dense(2))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 784, 32)           832       
                                                                 
 activation_17 (Activation)  (None, 784, 32)           0         
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 262, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_13 (Conv1D)          (None, 262, 64)           51264     
                                                                 
 activation_18 (Activation)  (None, 262, 64)           0         
                                                                 
 max_pooling1d_13 (MaxPoolin  (None, 88, 64)           0         
 g1D)                                                 

[0.03909013047814369, 0.6823827624320984, 1.0, 0.3176652193069458]

In [ ]:
N=x2_train_s.shape[0]
x_train=tf.reshape(x2_train_s,[N,28,28,1])
N2=x2_test_s.shape[0]
x_test=tf.reshape(x2_test_s,[N2,28,28,1])
y_train=y2_train_s
y_test=y2_test_s
model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(2))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 28, 28, 32)        832       
                                                                 
 activation_19 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 10, 10, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 10, 10, 64)        51264     
                                                                 
 activation_20 (Activation)  (None, 10, 10, 64)        0         
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                  

[0.03914114087820053,
 0.3176172375679016,
 1.8118865489959717,
 0.36761540174484253]

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x2_train_s.shape[0]
x_train=tf.reshape(x2_train_s,[N,28,28,1])
N2=x2_test_s.shape[0]
x_test=tf.reshape(x2_test_s,[N2,28,28,1])
y_train=y2_train_s
y_test=y2_test_s

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(2))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 32)        832       
                                                                 
 activation_21 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 10, 10, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 10, 10, 64)        51264     
                                                                 
 activation_22 (Activation)  (None, 10, 10, 64)        0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                 

[0.039149582386016846,
 0.3181242048740387,
 0.991892397403717,
 0.3180508315563202]

In [ ]:
##########download dataset################  SESSIONL7 2CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/2class/SessionL7
!pwd
x2_train_sl7 = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x2_test_sl7 = idx2numpy.convert_from_file('t10k-images-idx3-ubyte')
y2_train_sl7 = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y2_test_sl7 = idx2numpy.convert_from_file('t10k-labels-idx1-ubyte')
print(x2_train_sl7.shape)
print(x2_test_sl7.shape)
print(y2_train_sl7.shape)
print(y2_test_sl7.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/2class/SessionL7
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/2class/SessionL7
(29193, 28, 28)
(3244, 28, 28)
(29193,)
(3244,)


In [ ]:
############CUSTOM LOSS for 2 class###############
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=2
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/29193)
  
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x2_train_sl7.shape[0]
x_train=tf.reshape(x2_train_sl7,[N,784,1])
N2=x2_test_sl7.shape[0]
x_test=tf.reshape(x2_test_sl7,[N2,784,1])
y_train=y2_train_sl7
y_test=y2_test_sl7
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_14 (Conv1D)          (None, 784, 32)           832       
                                                                 
 activation_23 (Activation)  (None, 784, 32)           0         
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 262, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_15 (Conv1D)          (None, 262, 64)           51264     
                                                                 
 activation_24 (Activation)  (None, 262, 64)           0         
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, 88, 64)           0         
 g1D)                                                

[0.04631168395280838,
 0.6220715045928955,
 0.9495076537132263,
 0.3770202100276947]

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x2_train_sl7.shape[0]
x_train=tf.reshape(x2_train_sl7,[N,784,1])
N2=x2_test_sl7.shape[0]
x_test=tf.reshape(x2_test_sl7,[N2,784,1])
y_train=y2_train_sl7
y_test=y2_test_sl7
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
x=x_train
y=y_train
# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.0005)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_16 (Conv1D)          (None, 784, 32)           832       
                                                                 
 activation_25 (Activation)  (None, 784, 32)           0         
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 262, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_17 (Conv1D)          (None, 262, 64)           51264     
                                                                 
 activation_26 (Activation)  (None, 262, 64)           0         
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 88, 64)           0         
 g1D)                                                

[0.04648888483643532, 0.6220715045928955, 1.0, 0.37781813740730286]

In [ ]:
N=x2_train_sl7.shape[0]
x_train=tf.reshape(x2_train_sl7,[N,28,28,1])
N2=x2_test_sl7.shape[0]
x_test=tf.reshape(x2_test_sl7,[N2,28,28,1])
y_train=y2_train_sl7
y_test=y2_test_sl7
model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(2))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 28, 28, 32)        832       
                                                                 
 activation_27 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 10, 10, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 10, 10, 64)        51264     
                                                                 
 activation_28 (Activation)  (None, 10, 10, 64)        0         
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                 

[0.0464153066277504,
 0.42478421330451965,
 0.06260696053504944,
 0.39014652371406555]

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x2_train_sl7.shape[0]
x_train=tf.reshape(x2_train_sl7,[N,28,28,1])
N2=x2_test_sl7.shape[0]
x_test=tf.reshape(x2_test_sl7,[N2,28,28,1])
y_train=y2_train_sl7
y_test=y2_test_sl7

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(2))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
##########download dataset################  NOVPNFLOWALLLAYERS 6CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/6class/NovpnFlowAllLayers
!pwd
x6_train = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x6_test = idx2numpy.convert_from_file('t10k-images-idx3-ubyte')
y6_train = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y6_test = idx2numpy.convert_from_file('t10k-labels-idx1-ubyte')

print(x6_train.shape)
print(x6_test.shape)
print(y6_train.shape)
print(y6_test.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/NovpnFlowAllLayers
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/NovpnFlowAllLayers
(26792, 28, 28)
(2977, 28, 28)
(26792,)
(2977,)


In [ ]:
############CUSTOM LOSS for 2 class###############
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=6
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/26792)
  
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x6_train.shape[0]
x_train=tf.reshape(x6_train,[N,784,1])
N2=x6_test.shape[0]
x_test=tf.reshape(x6_test,[N2,784,1])
y_train=y6_train
y_test=y6_test
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.01)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_26 (Conv1D)          (None, 784, 32)           832       
                                                                 
 activation_37 (Activation)  (None, 784, 32)           0         
                                                                 
 max_pooling1d_26 (MaxPoolin  (None, 262, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_27 (Conv1D)          (None, 262, 64)           51264     
                                                                 
 activation_38 (Activation)  (None, 262, 64)           0         
                                                                 
 max_pooling1d_27 (MaxPoolin  (None, 88, 64)           0         
 g1D)                                                

[0.25365281105041504, 0.20154517889022827, 4.0, 0.7985802292823792]

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x6_train.shape[0]
x_train=tf.reshape(x6_train,[N,784,1])
N2=x6_test.shape[0]
x_test=tf.reshape(x6_test,[N2,784,1])
y_train=y6_train
y_test=y6_test
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.01)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_30 (Conv1D)          (None, 784, 32)           832       
                                                                 
 activation_41 (Activation)  (None, 784, 32)           0         
                                                                 
 max_pooling1d_30 (MaxPoolin  (None, 262, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_31 (Conv1D)          (None, 262, 64)           51264     
                                                                 
 activation_42 (Activation)  (None, 262, 64)           0         
                                                                 
 max_pooling1d_31 (MaxPoolin  (None, 88, 64)           0         
 g1D)                                                

[0.2535792887210846, 0.061807189136743546, 5.0, 0.7985802292823792]

In [ ]:
N=x6_train.shape[0]
x_train=tf.reshape(x6_train,[N,28,28,1])
N2=x6_test.shape[0]
x_test=tf.reshape(x6_test,[N2,28,28,1])
y_train=y6_train
y_test=y6_test
model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.01)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 28, 28, 32)        832       
                                                                 
 activation_45 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 10, 10, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 10, 10, 64)        51264     
                                                                 
 activation_46 (Activation)  (None, 10, 10, 64)        0         
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 4, 4, 64)         0         
 g2D)                                                

[0.2533799707889557, 0.20154517889022827, 1.0, 0.7985802292823792]

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x6_train.shape[0]
x_train=tf.reshape(x6_train,[N,28,28,1])
N2=x6_test.shape[0]
x_test=tf.reshape(x6_test,[N2,28,28,1])
y_train=y6_train
y_test=y6_test

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
##########download dataset################  VPNFLOWALLLAYERS 6CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/6class/VpnFlowAllLayers
!pwd
x6_train_vf = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x6_test_vf = idx2numpy.convert_from_file('t10k-images-idx3-ubyte')
y6_train_vf = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y6_test_vf = idx2numpy.convert_from_file('t10k-labels-idx1-ubyte')
print(x6_train_vf.shape)
print(x6_test_vf.shape)
print(y6_train_vf.shape)
print(y6_test_vf.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/VpnFlowAllLayers
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/VpnFlowAllLayers
(15030, 28, 28)
(1670, 28, 28)
(15030,)
(1670,)


In [ ]:
############CUSTOM LOSS for 2 class###############
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=6
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/15030)
  
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x6_train_vf.shape[0]
x_train=tf.reshape(x6_train_vf,[N,784,1])
N2=x6_test_vf.shape[0]
x_test=tf.reshape(x6_test_vf,[N2,784,1])
y_train=y6_train_vf
y_test=y6_test_vf
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_32 (Conv1D)          (None, 784, 32)           832       
                                                                 
 activation_47 (Activation)  (None, 784, 32)           0         
                                                                 
 max_pooling1d_32 (MaxPoolin  (None, 262, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_33 (Conv1D)          (None, 262, 64)           51264     
                                                                 
 activation_48 (Activation)  (None, 262, 64)           0         
                                                                 
 max_pooling1d_33 (MaxPoolin  (None, 88, 64)           0         
 g1D)                                                

[0.4503084719181061, 0.359281450510025, 2.136474847793579, 0.6275029182434082]

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x6_train_vf.shape[0]
x_train=tf.reshape(x6_train_vf,[N,784,1])
N2=x6_test_vf.shape[0]
x_test=tf.reshape(x6_test_vf,[N2,784,1])
y_train=y6_train_vf
y_test=y6_test_vf
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
x=x_train
y=y_train
# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.0005)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_34 (Conv1D)          (None, 784, 32)           832       
                                                                 
 activation_49 (Activation)  (None, 784, 32)           0         
                                                                 
 max_pooling1d_34 (MaxPoolin  (None, 262, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_35 (Conv1D)          (None, 262, 64)           51264     
                                                                 
 activation_50 (Activation)  (None, 262, 64)           0         
                                                                 
 max_pooling1d_35 (MaxPoolin  (None, 88, 64)           0         
 g1D)                                                

[0.4500720798969269, 0.359281450510025, 2.0, 0.6426470875740051]

In [ ]:
N=x6_train_vf.shape[0]
x_train=tf.reshape(x6_train_vf,[N,28,28,1])
N2=x6_test_vf.shape[0]
x_test=tf.reshape(x6_test_vf,[N2,28,28,1])
y_train=y6_train_vf
y_test=y6_test_vf

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 28, 28, 32)        832       
                                                                 
 activation_51 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 10, 10, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 10, 10, 64)        51264     
                                                                 
 activation_52 (Activation)  (None, 10, 10, 64)        0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 4, 4, 64)         0         
 g2D)                                                

[0.45144978165626526, 0.05508982017636299, 3.0, 0.6426470875740051]

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x6_train_vf.shape[0]
x_train=tf.reshape(x6_train_vf,[N,28,28,1])
N2=x6_test_vf.shape[0]
x_test=tf.reshape(x6_test_vf,[N2,28,28,1])
y_train=y6_train_vf
y_test=y6_test_vf

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 28, 28, 32)        832       
                                                                 
 activation_53 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 10, 10, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 10, 10, 64)        51264     
                                                                 
 activation_54 (Activation)  (None, 10, 10, 64)        0         
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 4, 4, 64)         0         
 g2D)                                                

[0.44794851541519165,
 0.057485029101371765,
 4.2641167640686035,
 0.6342486143112183]

In [ ]:
##########download dataset################  NONVPNFLOWL7 6CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/6class/NovpnFlowL7
!pwd
x6_train_nvfl7 = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x6_test_nvfl7 = idx2numpy.convert_from_file('test-images-idx3-ubyte')
y6_train_nvfl7 = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y6_test_nvfl7 = idx2numpy.convert_from_file('test-labels-idx1-ubyte')

print(x6_train_nvfl7.shape)
print(x6_test_nvfl7.shape)
print(y6_train_nvfl7.shape)
print(y6_test_nvfl7.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/NovpnFlowL7
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/NovpnFlowL7
(20444, 28, 28)
(2272, 28, 28)
(20444,)
(2272,)


In [ ]:
############CUSTOM LOSS for 2 class###############
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=6
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/20444)
  
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x6_train_nvfl7.shape[0]
x_train=tf.reshape(x6_train_nvfl7,[N,784,1])
N2=x6_test_nvfl7.shape[0]
x_test=tf.reshape(x6_test_nvfl7,[N2,784,1])
y_train=y6_train_nvfl7
y_test=y6_test_nvfl7
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.01)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_44 (Conv1D)          (None, 784, 32)           832       
                                                                 
 activation_63 (Activation)  (None, 784, 32)           0         
                                                                 
 max_pooling1d_44 (MaxPoolin  (None, 262, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_45 (Conv1D)          (None, 262, 64)           51264     
                                                                 
 activation_64 (Activation)  (None, 262, 64)           0         
                                                                 
 max_pooling1d_45 (MaxPoolin  (None, 88, 64)           0         
 g1D)                                                

[0.331290066242218, 0.30237674713134766, 4.058542251586914, 0.832527756690979]

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x6_train_nvfl7.shape[0]
x_train=tf.reshape(x6_train_nvfl7,[N,784,1])
N2=x6_test_nvfl7.shape[0]
x_test=tf.reshape(x6_test_nvfl7,[N2,784,1])
y_train=y6_train_nvfl7
y_test=y6_test_nvfl7
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
x=x_train
y=y_train
# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
#model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.0005)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_46 (Conv1D)          (None, 784, 32)           832       
                                                                 
 activation_65 (Activation)  (None, 784, 32)           0         
                                                                 
 max_pooling1d_46 (MaxPoolin  (None, 262, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_47 (Conv1D)          (None, 262, 64)           51264     
                                                                 
 activation_66 (Activation)  (None, 262, 64)           0         
                                                                 
 max_pooling1d_47 (MaxPoolin  (None, 88, 64)           0         
 g1D)                                                

[0.3299883306026459,
 0.12455985695123672,
 2.673790454864502,
 0.7959558963775635]

In [ ]:
N=x6_train_nvfl7.shape[0]
x_train=tf.reshape(x6_train_nvfl7,[N,28,28,1])
N2=x6_test_nvfl7.shape[0]
x_test=tf.reshape(x6_test_nvfl7,[N2,28,28,1])
y_train=y6_train_nvfl7
y_test=y6_test_nvfl7

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.01)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 28, 28, 32)        832       
                                                                 
 activation_67 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 10, 10, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 10, 10, 64)        51264     
                                                                 
 activation_68 (Activation)  (None, 10, 10, 64)        0         
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 4, 4, 64)         0         
 g2D)                                                

[-0.3648664653301239, 0.04577464610338211, 2.0, 0.8307510614395142]

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
##########download dataset################  VPNFLOWL7 6CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/6class/VpnFlowL7
!pwd
x6_train_vfl7 = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x6_test_vfl7 = idx2numpy.convert_from_file('t10k-images-idx3-ubyte')
y6_train_vfl7 = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y6_test_vfl7 = idx2numpy.convert_from_file('t10k-labels-idx1-ubyte')
print(x6_train_vfl7.shape)
print(x6_test_vfl7.shape)
print(y6_train_vfl7.shape)
print(y6_test_vfl7.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/VpnFlowL7
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/VpnFlowL7
(14407, 28, 28)
(1600, 28, 28)
(14407,)
(1600,)


In [ ]:
############CUSTOM LOSS for 2 class###############
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=6
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/14407)
  
  return custom_loss
  

In [ ]:
####### 1DCNN ########### No cross validation
N=x6_train_vfl7.shape[0]
x_train=tf.reshape(x6_train_vfl7,[N,784,1])
N2=x6_test_vfl7.shape[0]
x_test=tf.reshape(x6_test_vfl7,[N2,784,1])
y_train=y6_train_vfl7
y_test=y6_test_vfl7
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_48 (Conv1D)          (None, 784, 32)           832       
                                                                 
 activation_69 (Activation)  (None, 784, 32)           0         
                                                                 
 max_pooling1d_48 (MaxPoolin  (None, 262, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_49 (Conv1D)          (None, 262, 64)           51264     
                                                                 
 activation_70 (Activation)  (None, 262, 64)           0         
                                                                 
 max_pooling1d_49 (MaxPoolin  (None, 88, 64)           0         
 g1D)                                                

[0.4728408753871918, 0.375, 2.1014068126678467, 0.6125028133392334]

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x6_train_vfl7.shape[0]
x_train=tf.reshape(x6_train_vfl7,[N,784,1])
N2=x6_test_vfl7.shape[0]
x_test=tf.reshape(x6_test_vfl7,[N2,784,1])
y_train=y6_train_vfl7
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
#model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.0005)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
N=x6_train_vfl7.shape[0]
x_train=tf.reshape(x6_train_vfl7,[N,28,28,1])
N2=x6_test_vfl7.shape[0]
x_test=tf.reshape(x6_test_vfl7,[N2,28,28,1])
y_train=y6_train_vfl7
y_test=y6_test_vfl7
model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.01)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 28, 28, 32)        832       
                                                                 
 activation_73 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 10, 10, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 10, 10, 64)        51264     
                                                                 
 activation_74 (Activation)  (None, 10, 10, 64)        0         
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 4, 4, 64)         0         
 g2D)                                                

[0.4734596014022827, 0.375, 4.008072853088379, 0.6254695653915405]

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x6_train_vfl7.shape[0]
x_train=tf.reshape(x6_train_vfl7,[N,28,28,1])
N2=x6_test_vfl7.shape[0]
x_test=tf.reshape(x6_test_vfl7,[N2,28,28,1])
y_train=y6_train_vfl7
y_test=y6_test_vfl7

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
##########download dataset################  NONVPNSESSIONALLLAYERS 6CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/6class/NovpnSessionAllLayers
!pwd
x6_train_ns = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x6_test_ns = idx2numpy.convert_from_file('t10k-images-idx3-ubyte')
y6_train_ns = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y6_test_ns = idx2numpy.convert_from_file('t10k-labels-idx1-ubyte')
print(x6_train_ns.shape)
print(x6_test_ns.shape)
print(y6_train_ns.shape)
print(y6_test_ns.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/NovpnSessionAllLayers
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/NovpnSessionAllLayers
(24229, 28, 28)
(2692, 28, 28)
(24229,)
(2692,)


In [ ]:
############CUSTOM LOSS for 2 class###############
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=6
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/24229)
  
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x6_train_ns.shape[0]
x_train=tf.reshape(x6_train_ns,[N,784,1])
N2=x6_test_ns.shape[0]
x_test=tf.reshape(x6_test_ns,[N2,784,1])
y_train=y6_train_ns
y_test=y6_test_ns
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.01)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_52 (Conv1D)          (None, 784, 32)           832       
                                                                 
 activation_77 (Activation)  (None, 784, 32)           0         
                                                                 
 max_pooling1d_52 (MaxPoolin  (None, 262, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_53 (Conv1D)          (None, 262, 64)           51264     
                                                                 
 activation_78 (Activation)  (None, 262, 64)           0         
                                                                 
 max_pooling1d_53 (MaxPoolin  (None, 88, 64)           0         
 g1D)                                                

[0.28085553646087646, 0.22325408458709717, 4.0, 0.777001678943634]

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x6_train_ns.shape[0]
x_train=tf.reshape(x6_train_ns,[N,784,1])
N2=x6_test_ns.shape[0]
x_test=tf.reshape(x6_test_ns,[N2,784,1])
y_train=y6_train_ns
y_test=y6_test_ns
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
#model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.01)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
N=x6_train_ns.shape[0]
x_train=tf.reshape(x6_train_ns,[N,28,28,1])
N2=x6_test_ns.shape[0]
x_test=tf.reshape(x6_test_ns,[N2,28,28,1])
y_train=y6_train_ns
y_test=y6_test_ns

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 28, 28, 32)        832       
                                                                 
 activation_79 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 10, 10, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 10, 10, 64)        51264     
                                                                 
 activation_80 (Activation)  (None, 10, 10, 64)        0         
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 4, 4, 64)         0         
 g2D)                                                

[-0.27304911613464355,
 0.2615155875682831,
 3.385484218597412,
 0.7221484780311584]

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x6_train_ns.shape[0]
x_train=tf.reshape(x6_train_ns,[N,28,28,1])
N2=x6_test_ns.shape[0]
x_test=tf.reshape(x6_test_ns,[N2,28,28,1])
y_train=y6_train_ns
y_test=y6_test_ns

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
##########download dataset################  VPNSESSIONALLLAYERS 6CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/6class/VpnSessionAllLayers
!pwd
x6_train_vs = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x6_test_vs = idx2numpy.convert_from_file('t10k-images-idx3-ubyte')
y6_train_vs = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y6_test_vs = idx2numpy.convert_from_file('t10k-labels-idx1-ubyte')
print(x6_train_vs.shape)
print(x6_test_vs.shape)
print(y6_train_vs.shape)
print(y6_test_vs.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/VpnSessionAllLayers
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/VpnSessionAllLayers
(11272, 28, 28)
(1253, 28, 28)
(11272,)
(1253,)


In [ ]:
############CUSTOM LOSS for 2 class###############
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=6
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/11272)
  
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x6_train_vs.shape[0]
x_train=tf.reshape(x6_train_vs,[N,784,1])
N2=x6_test_vs.shape[0]
x_test=tf.reshape(x6_test_vs,[N2,784,1])
y_train=y6_train_vs
y_test=y6_test_vs
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_54 (Conv1D)          (None, 784, 32)           832       
                                                                 
 activation_81 (Activation)  (None, 784, 32)           0         
                                                                 
 max_pooling1d_54 (MaxPoolin  (None, 262, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_55 (Conv1D)          (None, 262, 64)           51264     
                                                                 
 activation_82 (Activation)  (None, 262, 64)           0         
                                                                 
 max_pooling1d_55 (MaxPoolin  (None, 88, 64)           0         
 g1D)                                                

[0.28084036707878113, 0.07821229100227356, 5.0, 0.686923086643219]

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x6_train_vs.shape[0]
x_train=tf.reshape(x6_train_vs,[N,784,1])
N2=x6_test_vs.shape[0]
x_test=tf.reshape(x6_test_vs,[N2,784,1])
y_train=y6_train_vs
y_test=y6_test_vs
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
#model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.01)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_56 (Conv1D)          (None, 784, 32)           832       
                                                                 
 activation_83 (Activation)  (None, 784, 32)           0         
                                                                 
 max_pooling1d_56 (MaxPoolin  (None, 262, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_57 (Conv1D)          (None, 262, 64)           51264     
                                                                 
 activation_84 (Activation)  (None, 262, 64)           0         
                                                                 
 max_pooling1d_57 (MaxPoolin  (None, 88, 64)           0         
 g1D)                                                

In [ ]:
N=x6_train_vs.shape[0]
x_train=tf.reshape(x6_train_vs,[N,28,28,1])
N2=x6_train_vs.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
##########download dataset################  NONVPNSESSIONL7 6CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/6class/NovpnSessionL7
!pwd
x6_train_nvsl7 = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x6_test_nvsl7 = idx2numpy.convert_from_file('t10k-images-idx3-ubyte')
y6_train_nvsl7 = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y6_test_nvsl7 = idx2numpy.convert_from_file('t10k-labels-idx1-ubyte')
print(x6_train_nvsl7.shape)
print(x6_test_nvsl7.shape)
print(y6_train_nvsl7.shape)
print(y6_test_nvsl7.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/NovpnSessionL7
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/NovpnSessionL7


In [ ]:
############CUSTOM LOSS for 2 class###############
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=6
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/41822)
  
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,784,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,784,1])
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,784,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,784,1])
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
x=x_train
y=y_train
# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
#model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.0005)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
##########download dataset################  VPNSESSIONL7 6CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/6class/VpnSessionL7
!pwd
x6_train_vsl7 = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x6_test_vsl7 = idx2numpy.convert_from_file('t10k-images-idx3-ubyte')
y6_train_vsl7 = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y6_test_vsl7 = idx2numpy.convert_from_file('t10k-labels-idx1-ubyte')
print(x6_train_vsl7.shape)
print(x6_test_vsl7.shape)
print(y6_train_vsl7.shape)
print(y6_test_vsl7.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/VpnSessionL7
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/6class/VpnSessionL7


In [ ]:
############CUSTOM LOSS for 2 class###############
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=6
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/41822)
  
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,784,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,784,1])
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,784,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,784,1])
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
x=x_train
y=y_train
# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
#model.add(Dropout(0.2))
model.add(Dense(6))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.0005)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(6))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
##########download dataset################  SESSIONL7 12CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/12class/SessionL7
!pwd
x12_train_sl7 = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x12_test_sl7 = idx2numpy.convert_from_file('test-images-idx3-ubyte')
y12_train_sl7 = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y12_test_sl7 = idx2numpy.convert_from_file('test-labels-idx1-ubyte')
print(x6_train_nvsl7.shape)
print(x6_test_nvsl7.shape)
print(y6_train_nvsl7.shape)
print(y6_test_nvsl7.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/12class/SessionL7
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/12class/SessionL7


In [ ]:
############CUSTOM LOSS for 2 class###############
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=12
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/41822)
  
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,784,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,784,1])
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(12))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,784,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,784,1])
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
x=x_train
y=y_train
# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
#model.add(Dropout(0.2))
model.add(Dense(12))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.0005)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(12))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(12))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
##########download dataset################  SESSIONALLLAYERS 12CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/12class/SessionAllLayers
!pwd
x12_train_s = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x12_test_s = idx2numpy.convert_from_file('test-images-idx3-ubyte')
y12_train_s = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y12_test_s = idx2numpy.convert_from_file('test-labels-idx1-ubyte')
print(x12_train_s.shape)
print(x12_test_s.shape)
print(y12_train_s.shape)
print(y12_test_s.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/12class/SessionAllLayers
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/12class/SessionAllLayers


In [ ]:
############CUSTOM LOSS for 2 class###############
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=12
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/41822)
  
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,784,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,784,1])
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(12))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,784,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,784,1])
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
x=x_train
y=y_train
# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
#model.add(Dropout(0.2))
model.add(Dense(12))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.0005)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(12))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(12))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
##########download dataset################  FLOWL7 12CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/12class/FlowL7
!pwd
x12_train_fl7 = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x12_test_fl7 = idx2numpy.convert_from_file('test-images-idx3-ubyte')
y12_train_fl7 = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y12_test_fl7 = idx2numpy.convert_from_file('test-labels-idx1-ubyte')
print(x12_train_fl7.shape)
print(x12_test_fl7.shape)
print(y12_train_fl7.shape)
print(y12_test_fl7.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/12class/FlowL7
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/12class/FlowL7


In [ ]:
############CUSTOM LOSS for 2 class###############
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=12
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/41822)
  
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,784,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,784,1])
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(12))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,784,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,784,1])
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
x=x_train
y=y_train
# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
#model.add(Dropout(0.2))
model.add(Dense(12))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.0005)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(12))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(12))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
##########download dataset################  FLOWALLLAYERS 12CLASS
%cd /content/drive/My\ Drive/$FOLDERNAME/2\.encrypted_traffic_classification/3\.PreprocessResults/12class/FlowAllLayers
!pwd
x12_train_f = idx2numpy.convert_from_file('train-images-idx3-ubyte')
x12_test_f = idx2numpy.convert_from_file('test-images-idx3-ubyte')
y12_train_f = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
y12_test_f = idx2numpy.convert_from_file('test-labels-idx1-ubyte')
print(x12_train_f.shape)
print(x12_test_f.shape)
print(y12_train_f.shape)
print(y12_test_f.shape)

/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/12class/FlowAllLayers
/content/drive/My Drive/Project/Project/DeepTraffic-master/2.encrypted_traffic_classification/3.PreprocessResults/12class/FlowAllLayers


In [ ]:
############CUSTOM LOSS for 2 class###############
import tensorflow.keras.backend as Kb
def custom_loss(y_true,y_pred):
  m = 0.35
  s = 30
  
  nrof_classes=12
  cos_theta = Kb.cos(y_pred)
  phi = cos_theta - m 
  label_onehot = tf.one_hot(y_true, nrof_classes)
  adjust_theta = s * tf.where(tf.equal(label_onehot,1), phi, cos_theta)
  right_exp_val=tf.where(tf.equal(label_onehot,1), Kb.exp(adjust_theta), 1)
  exp_val=Kb.exp(adjust_theta)
  loss_ind=0
  loss_ind = -Kb.sum(Kb.log(tf.divide(Kb.sum(right_exp_val,axis=1),Kb.sum(exp_val, axis=1))))
  custom_loss = tf.reduce_mean(loss_ind/41822)
  
  return custom_loss

In [ ]:
####### 1DCNN ########### No cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,784,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,784,1])
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(12))
print(model.summary())

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy", recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
#loss(y_train,y_test).numpy()
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
######## 1DCNN ########### 5 fold cross validation 
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,784,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,784,1])
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
x=x_train
y=y_train
# K-fold Cross Validation model evaluation
model= Sequential()
model.add(Conv1D(input_shape=(784,1), filters=32, kernel_size=(25), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Conv1D(filters=64,kernel_size=(25),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool1D(pool_size=(3), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
#model.add(Dropout(0.2))
model.add(Dense(12))
print(model.summary())

optim=tf.keras.optimizers.Adam(learning_rate=0.0005)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
fold_no = 1
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(12))
kfold = KFold(n_splits=5, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
for train, test in kfold.split(x_train, y_train):
  X_train=tf.gather(x_train,train)
  Y_train=tf.gather(y_train,train)
  X_test=tf.gather(x_train, test)
  Y_test =tf.gather(y_train,test)
  model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, verbose=2)
  model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
  fold_no = fold_no + 1
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)

In [ ]:
############## 2DCNN ###################### no fold cross validation
N=x_train.shape[0]
x_train=tf.reshape(x_train,[N,28,28,1])
N2=x_test.shape[0]
x_test=tf.reshape(x_test,[N2,28,28,1])

model= Sequential()
model.add(Conv2D(input_shape=(28,28,1), filters=32, kernel_size=(25,1), strides=1, padding='SAME'))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Conv2D(filters=64,kernel_size=(25,1),strides=1,padding="SAME"))
model.add(Activation("swish"))
model.add(MaxPool2D(pool_size=(3,1), strides=3, padding='SAME'))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
# dropout
#keep_prob = Kb.variable(dtype="float32")
model.add(Dropout(0.2))
model.add(Dense(12))
kfold = KFold(n_splits=10, shuffle=True)
print(model.summary())

print("y_train shape:",y_train.shape)

#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optim=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy",recall, precision]
model.compile(optimizer=optim, metrics=metrics,loss=custom_loss)
batch_size=50
epochs=5
model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs, verbose=2)
model.evaluate(x_test,y_test,batch_size=batch_size,verbose=2)